In [15]:
import mediapipe as mp
import pandas as pd
import numpy as np
import cv2
import os
import re


In [16]:
mp_pose = mp.solutions.pose


In [17]:
landmarks_points = {
    "nose": 0,
    "left_shoulder": 11, "right_shoulder": 12,
    "left_elbow": 13, "right_elbow": 14,
    "left_wrist": 15, "right_wrist": 16,
    "left_hip": 23, "right_hip": 24,
    "left_knee": 25, "right_knee": 26,
    "left_ankle": 27, "right_ankle": 28,
    "left_heel": 29, "right_heel": 30,
    "left_foot_index": 31, "right_foot_index": 32,
}

col_names = []
for i in range(len(landmarks_points.keys())):
    name = list(landmarks_points.keys())[i]
    col_names.append(name + "_x")
    col_names.append(name + "_y")
    col_names.append(name + "_z")
    col_names.append(name + "_v")

pose_name = col_names.copy()
step_name = col_names.copy()

pose_name.append("pose")  # from 0 to n
# col_names.append("correct") # correct: 0, wrong: 1
step_name.append("step")  # from 0 to n


In [18]:
pose_list = []

main_dir = "./poses_dataset/Images"
pose_dir_list = os.listdir(main_dir)

for i in range(0, len(pose_dir_list)):
    if i in [0, 2, 6, 7, 9]:
        continue
    step_list = []
    wrong_right_dir_list = os.listdir(f"{main_dir}/{pose_dir_list[i]}")

    for j in range(0, len(wrong_right_dir_list) - 1):
        steps_dir_list = os.listdir(
            f"{main_dir}/{pose_dir_list[i]}/{wrong_right_dir_list[j]}")

        for k in range(0, len(steps_dir_list)):
            images_dir_list = os.listdir(
                f"{main_dir}/{pose_dir_list[i]}/{wrong_right_dir_list[j]}/{steps_dir_list[k]}")
            if j == 1:
                reg = re.search(r"([0-9])", steps_dir_list[k])

            for l in range(0, len(images_dir_list)):
                pre_list = []
                with mp_pose.Pose(static_image_mode=True, enable_segmentation=True) as pose:
                    image = cv2.imread(
                        f"{main_dir}/{pose_dir_list[i]}/{wrong_right_dir_list[j]}/{steps_dir_list[k]}/{images_dir_list[l]}")
                    result = pose.process(
                        cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

                    try:
                        predict = True
                        for landmarks in result.pose_landmarks.landmark:
                            pre_list.append(landmarks)
                    except AttributeError:
                        print(
                            f"No points {main_dir}/{pose_dir_list[i]}/{wrong_right_dir_list[j]}/{steps_dir_list[k]}/{images_dir_list[l]}")
                        predict = False

                if predict == True:
                    gen1116 = np.array([
                        [
                            pre_list[m].x,
                            pre_list[m].y,
                            pre_list[m].z,
                            pre_list[m].visibility
                        ] for m in range(11, 17)
                    ]).flatten().tolist()

                    gen2333 = np.array([
                        [
                            pre_list[m].x,
                            pre_list[m].y,
                            pre_list[m].z,
                            pre_list[m].visibility
                        ] for m in range(23, 33)
                    ]).flatten().tolist()

                    gen1116.extend(gen2333)

                    all_list = [
                        pre_list[0].x,
                        pre_list[0].y,
                        pre_list[0].z,
                        pre_list[0].visibility,
                    ]

                    all_list.extend(gen1116)
                    tpl = all_list.copy()
                    tsl = all_list.copy()
                    tpl.append(i)
                    tsl.append(k)
                    pose_list.append(tpl)
                    step_list.append(tsl)
                    # all_list.append(i)
                    # all_list.append(j)
                    # if j == 1:
                    #     all_list.append(int(reg.group(1)))
                    # else:
                    #     all_list.append(k)
        # data_step = pd.DataFrame(step_list, columns=step_name)
        # data_step.to_csv(f"./csv_files/{pose_dir_list[i]}.csv", index=False)

data_pose = pd.DataFrame(pose_list, columns=pose_name)
data_pose.to_csv("./csv_files/data_pose.csv", index=False)


No points ./poses_dataset/Images/Ardhakati_Chakrasana/Ardhakati_Chakrasana_Right_Steps/Ardhakati_Chakrasana_Right_Step_1/20230311_083742.jpg
No points ./poses_dataset/Images/Marjariasana/Marjariasana_Right_Steps/Marjariasana_Right_Step_1/20230311_151821.jpg
No points ./poses_dataset/Images/Marjariasana/Marjariasana_Right_Steps/Marjariasana_Right_Step_1/20230311_151924.jpg
No points ./poses_dataset/Images/Marjariasana/Marjariasana_Right_Steps/Marjariasana_Right_Step_2/20230311_153402(0).jpg
No points ./poses_dataset/Images/Marjariasana/Marjariasana_Right_Steps/Marjariasana_Right_Step_2/20230311_154237.jpg
No points ./poses_dataset/Images/Marjariasana/Marjariasana_Right_Steps/Marjariasana_Right_Step_2/20230311_154240.jpg
No points ./poses_dataset/Images/Marjariasana/Marjariasana_Right_Steps/Marjariasana_Right_Step_3/20230311_155356.jpg
No points ./poses_dataset/Images/Marjariasana/Marjariasana_Right_Steps/Marjariasana_Right_Step_3/20230311_155409(0).jpg
No points ./poses_dataset/Images/M